## HamLib Simulation Observable Benchmark Tests - WIP

This notebook is being used to flesh out the details of integrating the HamLib utility module, the HamLib kernel function, and the newly developed functions for generating groups of commuting terms.

From any specific Hamiltonian, we create a base evolution circuit with 0 - N Trotter steps, initialize the circuit, and append the necessary basis rotations before measuring and computing the values of energy and other related observables.

At a later point, the execution code and plotting code will also be integrated into the benchmark framework. For now, however, the focus is on developing the code necessary to evaluate the performance of observable execution using various optimization options.


### Setup Dependencies

This cell is used to pull in dependencies from the benchmark framework, the HamLib utility module, and the newly created observables module (in WIP).

In [1]:
from qiskit.quantum_info import Pauli, SparsePauliOp
import numpy as np
from math import sin, cos, pi
import time

import sys
sys.path.insert(1, "WIP_benchmarks")

# Observable Helper Functions
from observables import *

sys.path[1:1] = ["../_common"]

import hamlib_simulation_kernel
from hamlib_simulation_kernel import HamiltonianSimulation, kernel_draw, get_valid_qubits
from hamlib_simulation_kernel import initial_state, create_circuit   # would like to remove these
from hamlib_utils import create_full_filenames, construct_dataset_name
from hamiltonian_simulation_exact import HamiltonianSimulationExact, HamiltonianSimulation_Noiseless

sys.path[1:1] = ["../../_common", "../../_common/qiskit"]

import execute

execute.verbose = False
execute.verbose_time = False


### Load HamLib Hamiltonian

Here we use the HamLib module to select a Hamiltonian from HamLib and initialize its options.  The API for these functions needs improvement.  The code here is taken from the current HamLib benchmark modules.

In [2]:
import hamlib_simulation_benchmark, hamlib_simulation_kernel, hamlib_utils

hamlib_simulation_benchmark.verbose = False
hamlib_simulation_kernel.verbose = False
hamlib_utils.verbose = False

hamlib_simulation_kernel.global_U = None
hamlib_simulation_kernel.global_enc = None
hamlib_simulation_kernel.global_ratio = None
hamlib_simulation_kernel.global_rinst = None
hamlib_simulation_kernel.global_h = None
hamlib_simulation_kernel.global_pbc_val = None

# get key infomation about the selected Hamiltonian
# DEVNOTE: Error handling here can be improved by simply returning False or raising exception
def get_hamiltonian_info(hamiltonian_name=None, init_state=None, K=None, t=None):
    try:
        hamlib_simulation_kernel.filename = create_full_filenames(hamiltonian_name)
        hamlib_simulation_kernel.dataset_name_template = construct_dataset_name(hamlib_simulation_kernel.filename)
    except ValueError:
        print(f"ERROR: cannot load HamLib data for Hamiltonian: {hamiltonian_name}")
        return
    
    if hamlib_simulation_kernel.dataset_name_template == "File key not found in data":
        print(f"ERROR: cannot load HamLib data for Hamiltonian: {hamiltonian_name}")
        return
    
    # Set default parameter values for the hamiltonians
    hamlib_simulation_kernel.set_default_parameter_values(hamlib_simulation_kernel.filename)
        
    # assume default init_state if not given
    if init_state == None:
        init_state = "checkerboard"


### Function to Create Circuits Directly or using Commuing Groups

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [3]:
# Create circuits optionally optimized by use of commuting groups
def create_circuits_for_hamiltonian(num_qubits, ham_terms, use_commuting_groups=True):

    # Create circuits from the Hamiltonian directly
    if not use_commuting_groups:   
        print("\n******** creating circuits from Hamiltonian:")
        circuits = create_circuits_ham(num_qubits, ham_terms)
    
    # Convert the Hamiltonian to groups and create the circuits from the groups
    else:   
        print("\n******** creating commuting groups for the Hamiltonian and circuits from the groups:")
        groups = group_commuting_terms_2(ham_terms)
        for i, group in enumerate(groups):
            print(f"Group {i+1}:")
            for pauli, coeff in group:
                print(f"  {pauli}: {coeff}")     
        circuits = create_circuits(num_qubits, groups)

    print(f"\n... constructed {len(circuits)} circuits for this Hamiltonian.")
    return circuits



### Create a Hamiltonian Simulation Circuit


In [13]:
# create the HamLibSimulation kernel, and the associated Hamiltonian operator

# Specify the desired Hamiltonian from HamLib 
# Set the hamiltonian_name: 'TFIM', 'Fermi-Hubbard-1D', 'Bose-Hubbard-1D', 'Heisenberg' or 'Max3Sat'
'''
hamiltonian_name = 'TFIM'
hamlib_simulation_kernel.global_h = 2
hamlib_simulation_kernel.global_pbc_val = 'pbc'
'''

hamiltonian_name = 'Fermi-Hubbard-1D'
hamlib_simulation_kernel.global_h = 2
hamlib_simulation_kernel.global_pbc_val = 'pbc'

'''
hamiltonian_name = 'Bose-Hubbard-1D'
hamlib_simulation_kernel.global_pbc_val = 'nonpbc'
hamlib_simulation_kernel.global_U = 10
hamlib_simulation_kernel.global_enc = 'gray'
'''

# convert SparsePauliOp to list form
ham_terms = ham_op.to_list()
print(ham_terms)

'''
###########################################

# classical simple Ising is ZZ
# TFIM ZZ + X  is transverse field
# + longitudinal field -> ZZ, X, and Z
def get_ising_hamiltonian(L, J, h, alpha=0):

    # List of Hamiltonian terms as 3-tuples containing
    # (1) the Pauli string,
    # (2) the qubit indices corresponding to the Pauli string,
    # (3) the coefficient.
    ZZ_tuples = [("ZZ", [i, i + 1], -J) for i in range(0, L - 1)]
    Z_tuples = [("Z", [i], -h * sin(alpha)) for i in range(0, L)]
    X_tuples = [("X", [i], -h * cos(alpha)) for i in range(0, L)]

    # We create the Hamiltonian as a SparsePauliOp, via the method
    # `from_sparse_list`, and multiply by the interaction term.
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *Z_tuples, *X_tuples], num_qubits=L)
    return hamiltonian.simplify()
    
H = get_ising_hamiltonian(L=num_qubits, J=0.2, h=1.2, alpha=pi / 8)
H_terms = H.to_list()

hamiltonian = H_terms

print(f"************ The test Hamiltonian")
print(hamiltonian)

ham_terms = hamiltonian
'''

init_state = "checkerboard"

get_hamiltonian_info(hamiltonian_name=hamiltonian_name, init_state=init_state)

# Parameters of simulation
num_qubits = 6
K = 1
t = 0.05
        
qc, bitstring, ham_op = HamiltonianSimulation(
    num_qubits, 
    K=K, t=t,
    hamiltonian = hamiltonian_name, 
    init_state = init_state,
    method = 1, 
)

print(qc)


[('XIIIII', (-0.5+0j)), ('XYYIII', (-0.5+0j)), ('XZIIII', (0.5+0j)), ('YYXIII', (0.5+0j)), ('ZYYIII', (-0.5+0j)), ('IXXIII', (-0.5+0j)), ('IZZYYX', (-0.5+0j)), ('IZZYIY', (-0.5+0j)), ('IIIXXX', (-0.5+0j)), ('IIIXZX', (-0.5+0j)), ('IIIIXI', (-0.5+0j)), ('IIIIXZ', (0.5+0j))]
************ The test Hamiltonian
[('IIIIZZ', (-0.2+0j)), ('IIIZZI', (-0.2+0j)), ('IIZZII', (-0.2+0j)), ('IZZIII', (-0.2+0j)), ('ZZIIII', (-0.2+0j)), ('IIIIIZ', (-0.4592201188381077+0j)), ('IIIIZI', (-0.4592201188381077+0j)), ('IIIZII', (-0.4592201188381077+0j)), ('IIZIII', (-0.4592201188381077+0j)), ('IZIIII', (-0.4592201188381077+0j)), ('ZIIIII', (-0.4592201188381077+0j)), ('IIIIIX', (-1.1086554390135441+0j)), ('IIIIXI', (-1.1086554390135441+0j)), ('IIIXII', (-1.1086554390135441+0j)), ('IIXIII', (-1.1086554390135441+0j)), ('IXIIII', (-1.1086554390135441+0j)), ('XIIIII', (-1.1086554390135441+0j))]
global phase: 0.125
        ┌───────────┐ ░  ┌─────────┐                                         »
   q_0: ┤ U3(π,0,π) ├

### Compute Energy of the Hamiltonian - Unoptimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [8]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = False

# Create circuits from the Hamiltonian 
circuits = create_circuits_for_hamiltonian(num_qubits, ham_terms, use_commuting_groups)

for circuit in circuits:
    print(circuit)
    print(circuit[0])

N = 3

print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
    ts = time.time()
    
    # Compile and execute the circuits
    transpiled_circuits = transpile([circuit for circuit, group in circuits], backend)

    print("")
    print(f"... transpilation time = {round(time.time()-ts, 3)}")
    
    # Execute all of the circuits to obtain array of result objects
    results = backend.run(transpiled_circuits).result()

    # Compute the total energy for the Hamiltonian
    total_energy = calculate_expectation(num_qubits, results, circuits, is_commuting=use_commuting_groups)

    print(f"... total execution time = {round(time.time()-ts, 3)}")
    print(f"Total Energy: {total_energy}")
    
print("")


******** creating circuits from Hamiltonian:
  ... IIIIZZ, (-0.2+0j)
  ... IIIZZI, (-0.2+0j)
  ... IIZZII, (-0.2+0j)
  ... IZZIII, (-0.2+0j)
  ... ZZIIII, (-0.2+0j)
  ... IIIIIZ, (-0.4592201188381077+0j)
  ... IIIIZI, (-0.4592201188381077+0j)
  ... IIIZII, (-0.4592201188381077+0j)
  ... IIZIII, (-0.4592201188381077+0j)
  ... IZIIII, (-0.4592201188381077+0j)
  ... ZIIIII, (-0.4592201188381077+0j)
  ... IIIIIX, (-1.1086554390135441+0j)
  ... IIIIXI, (-1.1086554390135441+0j)
  ... IIIXII, (-1.1086554390135441+0j)
  ... IIXIII, (-1.1086554390135441+0j)
  ... IXIIII, (-1.1086554390135441+0j)
  ... XIIIII, (-1.1086554390135441+0j)

... constructed 17 circuits for this Hamiltonian.
(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000002D141A06250>, [('IIIIZZ', (-0.2+0j))])
         ░ ┌─┐               
   q_0: ─░─┤M├───────────────
         ░ └╥┘┌─┐            
   q_1: ─░──╫─┤M├────────────
         ░  ║ └╥┘┌─┐         
   q_2: ─░──╫──╫─┤M├─────────
         ░  ║  ║ └╥┘┌─┐      
  

### Compute Energy of the Hamiltonian - Optimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [9]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = True

# Create circuits from the Hamiltonian 
circuits = create_circuits_for_hamiltonian(num_qubits, ham_terms, use_commuting_groups)

for circuit in circuits:
    print(circuit)
    print(circuit[0])

N = 3

print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
    ts = time.time()
    
    # Compile and execute the circuits
    transpiled_circuits = transpile([circuit for circuit, group in circuits], backend)

    print("")
    print(f"... transpilation time = {round(time.time()-ts, 3)}")
    
    # Execute all of the circuits to obtain array of result objects
    results = backend.run(transpiled_circuits).result()

    # Compute the total energy for the Hamiltonian
    total_energy = calculate_expectation(num_qubits, results, circuits, is_commuting=use_commuting_groups)

    print(f"... total execution time = {round(time.time()-ts, 3)}")
    print(f"Total Energy: {total_energy}")
    
print("")


******** creating commuting groups for the Hamiltonian and circuits from the groups:
Group 1:
  IIIIZZ: (-0.2+0j)
  IIIZZI: (-0.2+0j)
  IIZZII: (-0.2+0j)
  IZZIII: (-0.2+0j)
  ZZIIII: (-0.2+0j)
  IIIIIZ: (-0.4592201188381077+0j)
  IIIIZI: (-0.4592201188381077+0j)
  IIIZII: (-0.4592201188381077+0j)
  IIZIII: (-0.4592201188381077+0j)
  IZIIII: (-0.4592201188381077+0j)
  ZIIIII: (-0.4592201188381077+0j)
Group 2:
  IIIIIX: (-1.1086554390135441+0j)
  XIIIII: (-1.1086554390135441+0j)
  IIIIXI: (-1.1086554390135441+0j)
  IIIXII: (-1.1086554390135441+0j)
  IIXIII: (-1.1086554390135441+0j)
  IXIIII: (-1.1086554390135441+0j)

... constructed 2 circuits for this Hamiltonian.
(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000002D143DE8410>, [('IIIIZZ', (-0.2+0j)), ('IIIZZI', (-0.2+0j)), ('IIZZII', (-0.2+0j)), ('IZZIII', (-0.2+0j)), ('ZZIIII', (-0.2+0j)), ('IIIIIZ', (-0.4592201188381077+0j)), ('IIIIZI', (-0.4592201188381077+0j)), ('IIIZII', (-0.4592201188381077+0j)), ('IIZIII', (-0.459